In [ ]:
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import json
import cmasher as cmr
import seaborn as sns
import sys

from astropy.io import fits
from astropy.utils.misc import JsonCustomEncoder
import astropy.units as u

from turbustat.simulator import make_ppv
from turb_utils import make_extended, make_3dfield




3D maps

In [ ]:
r0 = 32.0
N = 256
m = 1.0

threeD_field = make_3dfield(
    N,
    ellip=0.5,
    theta=45,
    powerlaw=3.0 + m,
    randomseed=2021_10_08)

plt.figure(figsize=[20, 6])  
plt.subplot(131)  
plt.imshow(threeD_field.mean(0), origin='lower')  
plt.subplot(132)  
plt.imshow(threeD_field.mean(1), origin='lower')  
plt.subplot(133)  
plt.imshow(threeD_field.mean(2), origin='lower')  
plt.tight_layout()  

In [ ]:
threeD_field = make_3dfield(
    N,
    ellip=0.5,
    theta=45,
    correlation_length=r0,  
    powerlaw=3.0 + m,
    randomseed=2021_10_08)

plt.figure(figsize=[20, 6])  
plt.subplot(131)  
plt.imshow(threeD_field.mean(0), origin='lower')  
plt.subplot(132)  
plt.imshow(threeD_field.mean(1), origin='lower')  
plt.subplot(133)  
plt.imshow(threeD_field.mean(2), origin='lower')  
plt.tight_layout()  